## Recommender System - Marketplace Matching

In this notebook, we will: 
- Clean textual data from user-input verbatim posts
- Use a Word2Vec model to calculate document similarities 
- Sort the most similar user input to our training data in order to recommend similar products 
- Save this model in a format that allows us to refresh the testing data

The goal of this project is to create a recommender system to help Pangeans find the right "project" for them, given their profile information. Here, we are using legacy data that is from Pangea V2, when Pangeans were allowed to post services and requests, as well as purchase items on the platform. We are using the User-Inputted Titles to suggest similar services, or in V3, similar "projects". 

In [1]:
#Importing Libraries
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords') ###

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

import os, sys

from operator import add

from sklearn.preprocessing import normalize

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from adjustText import adjust_text

from sklearn.manifold import TSNE

import pickle

import json
from pprint import pprint
from pandas.io.json import json_normalize

import operator

import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelateng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [5]:
def vectorize_and_store_existing_titles():
    
    raw = pd.read_csv("allPostData.csv", header=0);
    
    #we can replace this with a filepath in the future
    titles = raw['title'];
    post_titles = [title for title in titles];
    post_titles = set(post_titles);
    
    tokens = [[word for word in title.lower().split()] for title in post_titles];
    
    clean_words = [[word for word in title if word.isalpha()] for title in tokens];
    stoplist = set(stopwords.words('english'));
    
    titles_nostopwords = [[word for word in title if word not in stoplist] for title in clean_words];   
    #print(len(titles_nostopwords))
    
    filtered_word_list = [[word for word in title if word in model.vocab] for title in titles_nostopwords];
    #print(len(filtered_word_list))
    
    dictionary = dict(zip(post_titles, filtered_word_list))
    
    title_vectors = {}
    
    #print(len(dictionary.keys()))
    
    #print(len(set(titles)))
    #print(len(set(post_titles)))
    #dupe reqs 
    
    vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
    print(vectorized_titles)
    
    
    
    for title in post_titles: 
        word_vecs = [model[word] for word in dictionary[title]]
        if len(word_vecs) == 0:
            title_vec = [np.zeros(300)]
        else: 
            title_vec = normalize(sum(word_vecs).reshape(1, -1))
       
        vectorized_titles = vectorized_titles.append({'Titles': title, 'Vectors': title_vec}, ignore_index=True)
       
    
    
    #dictionary = dict(zip(post_titles, filtered_word_list))

    #can also replace filepath in the future
    #titles.to_pickle("/Users/angelateng/Dropbox/SharpestMinds/raw_titles.pkl")
    vectorized_titles.to_pickle("/Users/angelateng/Dropbox/SharpestMinds/vectorized_titles.pkl")
    return(vectorized_titles)

#vectorize_and_store_existing_titles()


In [6]:
pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/vectorized_titles.pkl")
#sanity check
#vectorize_and_store_existing_titles()

,Titles,Vectors
0,hang up my TV mount,"[[-0.044453368, 0.044327557, -0.11491825, 0.04..."
1,Fantasy Football Consulting,"[[0.039403234, -0.016861744, 0.049298417, 0.00..."
2,Make Brandy Alexander Pie,"[[-0.072663724, -0.06542867, 0.011346995, 0.08..."
3,Photographer Wanted,"[[0.041132804, 0.055709932, 0.00022233947, -0...."
4,Soccer Lessons,"[[0.020405818, 0.049692925, 0.13152766, -0.036..."
5,Please Buy Me A Ben & Jerry’s Ice Cream Cake!!!,"[[-0.04161577, 0.041488077, 0.034238372, 0.043..."
6,Marketing Advice,"[[0.01688988, -0.03561996, -0.05343707, -0.020..."
7,Power Drill,"[[0.0013474241, 0.022457069, 0.057462025, -0.0..."
8,Software Engineer needed for small proj,"[[0.001173173, 0.013858106, -0.013720624, -0.0..."
9,Postering,"[[0.02030486, 0.011536853, 0.0025765637, 0.047..."


In [50]:
#can change this directory later


#test post 
with open('firstPost.json') as fresh_data:
    user_post = json.load(fresh_data)

#user_post = json.loads('firstPost.json')
#print(user_post)

#should only take in a single post at a time 

def vectorize_new_title(user_post):
    json_df = pd.DataFrame.from_dict(json_normalize(user_post), orient='columns')
    #print(json_df)
    
    title = json_df["title"]
    #print("this is the title", title)
    
    #json_post_titles = [title in title];
    json_post_titles = [title for title in title];
    json_tokens = [[word for word in title.lower().split()] for title in json_post_titles];
    #json_post_titles = set(json_post_titles)
    print(json_post_titles)
    
    
    
    #json_clean_words = [[word for word in title if word.isalpha()] for title in json_post_titles];
    json_clean_words = [[word for word in title if word.isalpha()] for title in json_tokens];
    print(json_clean_words)
    
    stoplist = set(stopwords.words('english'));
    json_titles_nostopwords = [[word for word in title if word not in stoplist] for title in json_clean_words];   
    
    
    json_filtered_word_list = [[word for word in title if word in model.vocab] for title in json_titles_nostopwords];
    print(json_filtered_word_list)
    
    json_title_vectors = {}
    
    json_vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
    print(json_vectorized_titles)
    
    
    for title in json_filtered_word_list: 
        json_word_vecs = [model[word] for word in title]
        if len(json_word_vecs) == 0:
            json_title_vec = [np.zeros(300)]
        else: 
            json_title_vec = normalize(sum(json_word_vecs).reshape(1, -1))
        
        json_vectorized_titles = json_vectorized_titles.append({'Titles': title, 'Vectors': json_title_vec}, ignore_index=True)
   

    return(json_vectorized_titles)

#vectorize_new_title(json_df)

In [51]:
vectorize_new_title(user_post)

['Teach Me How To Cook!']
[['teach', 'me', 'how', 'to']]
[['teach']]
Empty DataFrame
Columns: [Titles, Vectors]
Index: []


,Titles,Vectors
0,[teach],"[[0.042356346, -0.013683017, 0.07774045, 0.028..."


In [9]:
vectorized_title = vectorize_new_title(user_post)

def rank_existing_titles(vectorized_title):
    #loop over all keys in dict 
    ranked_titles = {}
    other_titles = pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/raw_titles.pkl")
    #other_titles = pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/vectorized_titles.pkl")
    #can also use title_vectors.keys() 
    for title in other_titles:
        #ranked_titles[title] = np.dot(other_titles[title], vectorized_title)[0]
        ranked_titles[title] = np.dot(other_titles, vectorized_title)[0]
    #print(other_titles)
        #print(np.dot(other_titles[title], vectorized_title))
    sorted_title_vecs = sorted(ranked_titles.items(), key=operator.itemgetter(1), reverse=True)
    #print(sorted_title_vecs)
    #dicts not ordered 
    return(sorted_title_vecs)

#rank_existing_titles(vectorized_title[:10])
#word querying

In [10]:
rank_existing_titles(vectorized_title)

ValueError: shapes (811,) and (300,1) not aligned: 811 (dim 0) != 300 (dim 0)

In [ ]:
other_titles = pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/raw_titles.pkl")
other_titles


In [ ]:
vectorized_title = vectorize_new_title(user_post)
print(vectorized_title)

In [ ]:
def generate_recommendations(user_post):
    title = user_post["title"]
    print(title)
    vectorized_title = vectorize_new_title(user_post)
    print(vectorized_title)
    ranked_titles = rank_existing_titles(title, vectorized_title)
    print(ranked_titles)
    
    
    with open("/Users/angelateng/Dropbox/SharpestMinds/ranked_titles.csv", "w", newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for title in ranked_titles:
            wr.writerow([ranked_titles, title])
    #csv_out = ranked_titles.to_csv('ranked_titles', encoding='utf-8', index=False)
    #writer = csv.writer(ranked_titles, delimiter='', quotechar='"', quoting=csv.QUOTE_ALL)
 
    #for row in reader:
        #writer.writerow(row)

       # with open('csv_out', 'wb') as csv_out:
         #   csv_writer = csv.writer(csv_out, quoting=csv.QUOTE_ALL)
         #   csv_writer.writerow('csv_out')
    return(ranked_titles)
    
    

#vectorize_and_store_existing_titles(); 
#vectorize_new_title(json_df);
#rank_existing_titles(vectorize_new_title);

In [ ]:
#generate_recommendations(json_df)
pprint(generate_recommendations(user_post))

In [ ]:
vectorize_new

In [ ]:
print(other_titles)